## Building a Spam Filter using Naive Bayes

In this guided project, we're going to study the practical side of the algorithm by building a spam filter for SMS messages.

To classify messages as spam or non-spam, we saw in the previous mission that the computer:

   1\. Learns how humans classify messages.
   2\. Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
  3\.  Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).


So our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly from this [link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.


## Explorting the Dataset

In [22]:
import pandas as pd
import numpy as np

df = pd.read_csv('SMSSpamCollection', sep='\t',header=None,names=['Label', 'SMS'])

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [24]:
df.head(10)

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [25]:
df['Label'].value_counts("ham")*100


ham     86.593683
spam    13.406317
Name: Label, dtype: float64

## Training and Test Set
We're now going to split our dataset into a training and a test set, where the training set accounts for 80% of the data, and the test set for the remaining 20%.

In [26]:
#random shuffle
df_rand = df.sample(frac = 1, random_state=1)

#spltting 80-20
train = df_rand[0:4457]
test = df_rand[4458:5572]

#resetting indexes
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)



We'll now analyze the percentage of spam and ham messages in the training and test sets. We expect the percentages to be close to what we have in the full dataset, where about 87% of the messages are ham, and the remaining 13% are spam.

In [27]:
train['Label'].value_counts("ham")*100

ham     86.53803
spam    13.46197
Name: Label, dtype: float64

In [28]:
test['Label'].value_counts("ham")*100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

## Data Cleaning
To calculate all the probabilities required by the algorithm, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

In [29]:
train.head(10)


,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...
5,ham,Ok i thk i got it. Then u wan me 2 come now or...
6,ham,I want kfc its Tuesday. Only buy 2 meals ONLY ...
7,ham,No dear i was sleeping :-P
8,ham,Ok pa. Nothing problem:-)
9,ham,Ill be there on &lt;#&gt; ok.


In [30]:
train['SMS'] = train['SMS'].str.replace('\W',' ')
train['SMS'] = train['SMS'].str.lower()

train.head(10)

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...
5,ham,ok i thk i got it then u wan me 2 come now or...
6,ham,i want kfc its tuesday only buy 2 meals only ...
7,ham,no dear i was sleeping p
8,ham,ok pa nothing problem
9,ham,ill be there on lt gt ok


## Creating the Vocabulary

Transform the vocabulary list into a set using the set() function. This will remove the duplicates from the vocabulary list.
Transform the vocabulary set back into a list using the list() function.

In [31]:
vocabulary = []

train['SMS']=train['SMS'].str.split()

for sms in train['SMS']:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))

In [32]:
len(vocabulary)

#vocabulary

7782

## The Final Training Set
We're now going to use the vocabulary we just created to make the data transformation we want.

In [33]:
word_counts_per_sms = {unique_word: [0] * len(train['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        


In [34]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,handset,id,cnupdates,connected,neway,manege,tacos,messaging,graphics,cochin,...,question,golddigger,hanuman,bash,im,brah,ubandu,connection,spam,bf
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
train_clean = pd.concat([train, word_counts], axis = 1)

train_clean.head()

,Label,SMS,handset,id,cnupdates,connected,neway,manege,tacos,messaging,...,question,golddigger,hanuman,bash,im,brah,ubandu,connection,spam,bf
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##  Calculating Constants First

![Calc_constants.png]()

Some of the terms in the four equations above will have the same value for every new message. As a start, let's first calculate:

P(Spam) and P(Ham)
NSpam, NHam, NVocabulary

In [36]:
spam_perc = train_clean ['Label'].value_counts()

n_spam = spam_perc["spam"]
n_ham = spam_perc["ham"]

## Calculating Parameters
Now that we have the constant terms calculated above, we can move on with calculating the parameters. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

In [37]:

#p_spam and p_ham
p_spam = spam_perc["spam"]/len(train_clean)

p_ham = spam_perc["ham"]/len(train_clean)

# N_Vocabulary
n_vocabulary = len(vocabulary)

print('prob of spam is: ', p_spam)
print('prob of ham is: ', p_ham)
print('n_vocabulary is: ', n_vocabulary)


prob of spam is:  0.13461969934933812
prob of ham is:  0.8653803006506618
n_vocabulary is:  7782


In [18]:
alpha = 1

spam_msg = train_clean[train_clean["Label"]=="spam"]

ham_msg = train_clean[train_clean["Label"]=="ham"]

# N_Spam
n_words_per_spam_message = spam_msg['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_msg['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()


In [19]:
n_ham

57233

In [20]:
n_spam

15190

## Calculating Parameters

On the previous screen, we managed to calculate a few terms for our equations:

P(Spam) and P(Ham)
NSpam, NHam, NVocabulary
As we've already mentioned, all these terms will have constant values in our equations for every new message (regardless of the message or each individual word in the message).

However, P(wi|Spam) and P(wi|Ham) will vary depending on the individual words. For instance, P("secret"|Spam) will have a certain probability value, while P("cousin"|Spam) or P("lovely"|Spam) will most likely have other values.

Although both P(wi|Spam) and P(wi|Ham) vary depending on the word, the probability for each individual word is constant for every new message.

For instance, let's say we receive two new messages:

"secret code"
"secret party 2night"
We'll need to calculate P("secret"|Spam) for both these messages, and we can use the training set to get the values we need to find a result for the equation below:

In more technical language, the probability values that P(wi|Spam) and P(wi|Ham) will take are called parameters.

The fact that we calculate so many values before even beginning the classification of new messages makes the Naive Bayes algorithm very fast (especially compared to other algorithms). When a new message comes in, most of the needed computations are already done, which enables the algorithm to almost instantly classify the new message.

If we didn't calculate all these values beforehand, then all these calculations would need to be done every time a new message comes in. Imagine the algorithm will be used to classify 1,000,000 new messages. Why repeat all these calculations 1,000,000 times when we could just do them once at the beginning?

In [21]:
key_list = vocabulary
y = 0


#theta or parameters for ham and spam
param_ham = dict.fromkeys(key_list,y)  
param_spam = dict.fromkeys(key_list,y)



In [22]:
for word in vocabulary:
    n_word_given_spam = spam_msg[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    param_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_msg[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    param_ham[word] = p_word_given_ham

In [23]:
param_ham

{'contented': 3.076213181573483e-05,
 '09050001295': 1.5381065907867414e-05,
 'accumulation': 3.076213181573483e-05,
 'dasara': 3.076213181573483e-05,
 'holder': 1.5381065907867414e-05,
 'cashto': 1.5381065907867414e-05,
 'reaching': 0.00013842959317080674,
 'odi': 3.076213181573483e-05,
 'aluable': 3.076213181573483e-05,
 'is': 0.008921018226563101,
 'hands': 7.690532953933707e-05,
 'mnths': 1.5381065907867414e-05,
 'order': 0.0001076674613550719,
 'around': 0.0007690532953933708,
 'fixed': 0.0001076674613550719,
 '09061104276': 1.5381065907867414e-05,
 'n8': 3.076213181573483e-05,
 'worries': 0.00015381065907867414,
 'rates': 3.076213181573483e-05,
 'teach': 9.228639544720449e-05,
 'abnormally': 3.076213181573483e-05,
 'persevered': 3.076213181573483e-05,
 'msgrcvd18': 1.5381065907867414e-05,
 'tacos': 4.614319772360224e-05,
 'dogwood': 3.076213181573483e-05,
 'essay': 3.076213181573483e-05,
 '021': 1.5381065907867414e-05,
 'black': 0.0001076674613550719,
 'inst': 3.076213181573483e-

## Classifying A New Message


Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
- If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
- If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
- If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [24]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

        
         # This is where we calculate:
        ## innitiate the values for probabilities
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # iterating over each word of the message
    
    for word in message:
        if word in param_spam:
            p_spam_given_message *= param_spam[word]
        if word in param_ham:
            p_spam_given_message *= param_ham[word]
              

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [25]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 3.02108770747198e-52
P(Ham|message): 0.8653803006506618
Label: Ham


In [26]:
classify ("Sounds good, Tom, then see u there")

P(Spam|message): 1.0396205861150208e-45
P(Ham|message): 0.8653803006506618
Label: Ham


On the previous screen, we managed to create a spam filter, and we classified two new messages. We'll now try to determine how well the spam filter does on our test set of 1,114 messages.

The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label (given by a human). Note that, in training, our algorithm didn't see these 1,114 messages, so every message in the test set is practically new from the perspective of the algorithm.

Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:

## Measuring the Spam Filter's Accuracy
The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.
=
number of correctly classified messages/total number of classified messages


In [27]:
def classify_test_set(message):    
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in param_spam:
            p_spam_given_message *= param_spam[word]
            
        if word in param_ham:
            p_ham_given_message *= param_ham[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [28]:
test['predicted'] = test['SMS'].apply(classify_test_set)
test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


What do you think about the accuracy value? Is it better or worse than you expected?

In [29]:
correct = 0
total = test.shape[0]
    
for row in test.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


The accuracy is close to 98.74%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.